# PolyLogToolkit

*LogToolkit* is a script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

#### Tensor $\otimes$

#### Wedge $\wedge$

#### Differential $d$

Elements in $\mathbb I^{\text{Symb}}$

$$
I(a_{i_0}; 0^{n_0-1}, a_{i_1}, 0^{n_1-1}, \cdots, a_{i_m}, 0^{m_d-1}; a_{i_{m+1}})\leftrightsquigarrow (i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})
$$

for example, in depth $2$, $I(a_0; 0^{1-1}, a_1, 0^{3-1}, a_2, 0^{2-1}; a_3)\leftrightsquigarrow(0,1,1,3,2,2,2)$

Recall there is a total ordering on the generators $\mathbb H^{\text{Symb}}$ by encoding

$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow(0^{i_1-1},n_1,\cdots,0^{i_d-i_{d-1}-1},n_d,0^{i_{d+1}-i_d-1})
$$

We regard $(0^{m_1},n_1,\cdots,0^{m_d},n_d,0^{m_{d+1}})$ as $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ in the implementation.

In [64]:
from sympy import *
II = IndexedBase('II')
Li = IndexedBase('Li')

In [191]:
class ISymb:
    def __new__(cls, *args):
        return II[args]
    def __repr__(self):
        return f'I{self.args}'

In [197]:
class HSymb:
    def __new__(cls, *args):
        return Li[args]
    def __repr__(self):
        return f'Li{self.args}'

In [ ]:
class tensor:
    def __init__(self, *comp):
        self.comp = comp

In [188]:
(ISymb(1,2,3).args)

(I, 1, 2, 3)

In [200]:
ISymb(1) * ISymb(2) + ISymb(2,3,4) ** 2

I[1]*I[2] + I[2, 3, 4]**2

In [212]:
(Rational(2, 3) * ISymb(1) * (1 + 2 * I)).args

NameError: name 'Complex' is not defined

In [143]:
(ISymb(IGen(1)) * ISymb(IGen(2)) + ISymb(IGen(2,3,4)) ** 2).degree()

1

In [142]:
(ISymb(IGen(1)) * ISymb(IGen(2)) + ISymb(IGen(2,3,4)) ** 2).terms()

[((1, 1, 0), 1), ((0, 0, 2), 1)]

In [141]:
ISymb(IGen(1)) * ISymb(IGen(2)) + ISymb(IGen(2,3,4)) ** 2

ISymb(I[1]*I[2] + I[2, 3, 4]**2, I[1], I[2], I[2, 3, 4], domain='QQ')

In [146]:
(Poly(I[0]*I[1])).degree()

1

In [160]:
Expr

type

In [159]:
isinstance(I[1]*I[0]+I[2]**2, Expr)

True